In [ ]:
!cp ../input/santa2020lgbmmodels/gbm_0125_32.pkl gbm_0125_32.pkl
!cp ../input/santa2020lgbmmodels/gbm_0131_32_quantile60.pkl gbm_0131_32_quantile60.pkl
!cp ../input/santa2020lgbmmodels/gbm_0131_32_quantile65.pkl gbm_0131_32_quantile65.pkl

In [ ]:
%%writefile lgbm_quantile.py

import numpy as np
import pandas as pd
import copy


model = None
model60 = None
cnt = [[0] * 100, [0] * 100]
cntd = [[0] * 100, [0] * 100]
hit = [[0] * 100, [0] * 100]
hitd = [[0] * 100, [0] * 100]
last = [[-1] * 100, [-1] * 100]
cnt_all = [0, 0]
cntd_all = [0, 0]
la = -1


def div(numer, denom):
    if denom == 0:
        return 0
    else:
        return numer / denom


def set_la(last_action):
    global la
    la = last_action


def agent(obs, conf):

    global model, model60, cnt, cntd, hit, hitd, last, cnt_all, cntd_all, la

    if model is None:
        model = pd.read_pickle("gbm_0131_32_quantile65.pkl")
        model60 = pd.read_pickle("gbm_0131_32_quantile60.pkl")

    ti = obs["step"]

    if ti == 0:
        cnt = [[0] * 100, [0] * 100]
        cntd = [[0] * 100, [0] * 100]
        hit = [[0] * 100, [0] * 100]
        hitd = [[0] * 100, [0] * 100]
        last = [[-1] * 100, [-1] * 100]
        cnt_all = [0, 0]
        cntd_all = [0, 0]
        la = -1
        # pass
    else:
        ola = sum(obs['lastActions']) - la
        disc1 = pow(0.97, cnt[0][la] + cnt[1][la])
        disc2 = pow(0.97, cnt[0][ola] + cnt[1][ola])
        inv1 = min(100000.0, 1.0 / disc1)
        inv2 = min(100000.0, 1.0 / disc2)

        last[0][la] = ti  # - 1
        last[1][ola] = ti  # - 1
        if sum(hit[0]) < obs['reward']:
            hit[0][la] += 1
            hitd[0][la] += inv1
        cnt[0][la] += 1
        cntd[0][la] += inv1
        cnt_all[0] += 1
        cntd_all[0] += inv1
        cnt[1][ola] += 1
        cntd[1][ola] += inv2
        cnt_all[1] += 1
        cntd_all[1] += inv2

    # print(cntd_all)

    ti += 1

    fs = []
    cnt1sorted = sorted(copy.deepcopy(cnt[0]))
    cnt1dsorted = sorted(copy.deepcopy(cntd[0]))
    cnt2sorted = sorted(copy.deepcopy(cnt[1]))
    cnt2dsorted = sorted(copy.deepcopy(cntd[1]))
    hit1sorted = sorted(copy.deepcopy(hit[0]))
    hit1dsorted = sorted(copy.deepcopy(hitd[0]))
    last1sorted = sorted([ti - last[0][x] for x in range(100)])
    last2sorted = sorted([ti - last[1][x] for x in range(100)])

    discs = [0.97 ** (cnt[0][k] + cnt[1][k]) for k in range(100)]

    for i in range(100):
        disc = discs[i]
        f = list()
        pb = f.append

        pb(ti)

        pb(cnt[0][i])
        pb(cnt1sorted.index(f[-1]))
        pb(cnt_all[0] - cnt[0][i])
        pb(div(cnt_all[0] - cnt[0][i], cnt_all[0]))
        pb(div(cnt[0][i], cnt_all[0]))

        pb(cntd[0][i])
        pb(cnt1dsorted.index(f[-1]))
        pb(cntd_all[0] - cntd[0][i])
        pb(div(cntd_all[0] - cntd[0][i], cntd_all[0]))
        pb(div(cntd[0][i], cntd_all[0]))

        pb(cnt[1][i])
        pb(cnt2sorted.index(f[-1]))
        pb(cnt_all[1] - cnt[1][i])
        pb(div(cnt_all[1] - cnt[1][i], cnt_all[1]))
        pb(div(cnt[1][i], cnt_all[1]))

        pb(cntd[1][i])
        pb(cnt2dsorted.index(f[-1]))
        pb(cntd_all[1] - cntd[1][i])
        pb(div(cntd_all[1] - cntd[1][i], cntd_all[1]))
        pb(div(cntd[1][i], cntd_all[1]))

        pb(hit[0][i])
        pb(hit1sorted.index(f[-1]))
        pb(hitd[0][i])
        pb(hit1dsorted.index(f[-1]))
        pb(div(hit[0][i], cnt[0][i]))
        pb(div(hitd[0][i], cnt[0][i]))

        pb(ti-last[0][i])
        pb(last1sorted.index(f[-1]))

        pb(ti-last[1][i])
        pb(last2sorted.index(f[-1]))

        pb(disc)

        fs.append(f)

    # in_feature = fs
    if ti < 750:
        thresh = model60.predict(fs)
    else:
        thresh = model.predict(fs)

    for i in range(100):
        # disc = 0.97 ** (cnt[0][i] + cnt[1][i])
        thresh[i] *= discs[i]

    # print(thresh)
    action = np.argmax(thresh)

    if ti == 1:
        action = np.random.randint(0, 100)

    la = action
    return int(action)


In [ ]:
%%writefile lgbm_regression.py

import numpy as np
import pandas as pd
import copy


model = None
cnt = [[0] * 100, [0] * 100]
cntd = [[0] * 100, [0] * 100]
hit = [[0] * 100, [0] * 100]
hitd = [[0] * 100, [0] * 100]
last = [[-1] * 100, [-1] * 100]
cnt_all = [0, 0]
cntd_all = [0, 0]
la = -1


def div(numer, denom):
    if denom == 0:
        return 0
    else:
        return numer / denom


def set_la(last_action):
    global la
    la = last_action


def agent(obs, conf):

    global model, cnt, cntd, hit, hitd, last, cnt_all, cntd_all, la

    if model is None:
        model = pd.read_pickle("gbm_0125_32.pkl")

    ti = obs["step"]

    if ti == 0:
        cnt = [[0] * 100, [0] * 100]
        cntd = [[0] * 100, [0] * 100]
        hit = [[0] * 100, [0] * 100]
        hitd = [[0] * 100, [0] * 100]
        last = [[-1] * 100, [-1] * 100]
        cnt_all = [0, 0]
        cntd_all = [0, 0]
        la = -1
        # pass
    else:
        ola = sum(obs['lastActions']) - la
        disc1 = pow(0.97, cnt[0][la] + cnt[1][la])
        disc2 = pow(0.97, cnt[0][ola] + cnt[1][ola])
        inv1 = min(100000.0, 1.0 / disc1)
        inv2 = min(100000.0, 1.0 / disc2)

        last[0][la] = ti  # - 1
        last[1][ola] = ti  # - 1
        if sum(hit[0]) < obs['reward']:
            hit[0][la] += 1
            hitd[0][la] += inv1
        cnt[0][la] += 1
        cntd[0][la] += inv1
        cnt_all[0] += 1
        cntd_all[0] += inv1
        cnt[1][ola] += 1
        cntd[1][ola] += inv2
        cnt_all[1] += 1
        cntd_all[1] += inv2

    # print(cntd_all)

    ti += 1

    fs = []
    cnt1sorted = sorted(copy.deepcopy(cnt[0]))
    cnt1dsorted = sorted(copy.deepcopy(cntd[0]))
    cnt2sorted = sorted(copy.deepcopy(cnt[1]))
    cnt2dsorted = sorted(copy.deepcopy(cntd[1]))
    hit1sorted = sorted(copy.deepcopy(hit[0]))
    hit1dsorted = sorted(copy.deepcopy(hitd[0]))
    last1sorted = sorted([ti - last[0][x] for x in range(100)])
    last2sorted = sorted([ti - last[1][x] for x in range(100)])

    discs = [0.97 ** (cnt[0][k] + cnt[1][k]) for k in range(100)]

    for i in range(100):
        disc = discs[i]
        f = list()
        pb = f.append

        pb(ti)

        pb(cnt[0][i])
        pb(cnt1sorted.index(f[-1]))
        pb(cnt_all[0] - cnt[0][i])
        pb(div(cnt_all[0] - cnt[0][i], cnt_all[0]))
        pb(div(cnt[0][i], cnt_all[0]))

        pb(cntd[0][i])
        pb(cnt1dsorted.index(f[-1]))
        pb(cntd_all[0] - cntd[0][i])
        pb(div(cntd_all[0] - cntd[0][i], cntd_all[0]))
        pb(div(cntd[0][i], cntd_all[0]))

        pb(cnt[1][i])
        pb(cnt2sorted.index(f[-1]))
        pb(cnt_all[1] - cnt[1][i])
        pb(div(cnt_all[1] - cnt[1][i], cnt_all[1]))
        pb(div(cnt[1][i], cnt_all[1]))

        pb(cntd[1][i])
        pb(cnt2dsorted.index(f[-1]))
        pb(cntd_all[1] - cntd[1][i])
        pb(div(cntd_all[1] - cntd[1][i], cntd_all[1]))
        pb(div(cntd[1][i], cntd_all[1]))

        pb(hit[0][i])
        pb(hit1sorted.index(f[-1]))
        pb(hitd[0][i])
        pb(hit1dsorted.index(f[-1]))
        pb(div(hit[0][i], cnt[0][i]))
        pb(div(hitd[0][i], cnt[0][i]))

        pb(ti-last[0][i])
        pb(last1sorted.index(f[-1]))

        pb(ti-last[1][i])
        pb(last2sorted.index(f[-1]))

        pb(disc)

        fs.append(f)

    # in_feature = fs
    thresh = model.predict(fs)
    for i in range(100):
        # disc = 0.97 ** (cnt[0][i] + cnt[1][i])
        thresh[i] *= discs[i]

    # print(thresh)
    action = np.argmax(thresh)

    if ti == 1:
        action = np.random.randint(0, 100)

    la = action
    return int(action)


In [ ]:
! pip install -U kaggle-environments

In [ ]:
from kaggle_environments import make

In [ ]:
env = make("mab")

In [ ]:
res = env.run(["lgbm_quantile.py", "lgbm_regression.py"])

In [ ]:
res[-1]

In [ ]:
from IPython.display import Image
Image("../input/santa2020fig/score_submission.png")

In [ ]:
Image("../input/santa2020fig/evaluation.png")